In [1]:
import tensorflow as tf
import keras
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras
keras.backend.get_session().run(tf.global_variables_initializer())  

Using TensorFlow backend.
W0827 22:47:24.168667 140658272417600 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0827 22:47:24.169420 140658272417600 deprecation_wrapper.py:119] From /home/mindy/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:207: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
import warnings
warnings.filterwarnings('ignore')

In [3]:
# load train and test df for labels 
train_df = pd.read_pickle('finished_pickles/train_df_glasses.pkl')
test_df = pd.read_pickle('finished_pickles/test_df_glasses.pkl')

In [35]:
train_data = np.load(open('bottleneck_model_weights/bottleneck_features_train.npy','rb'))
train_label = np.array(train_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))
    
test_data = np.load(open('bottleneck_model_weights/bottleneck_features_test.npy','rb'))
test_label = np.array(test_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))[:-3]  # didn't run all 131 samples 

In [50]:
train_label[:-10].shape

(512,)

In [53]:
def train_top_model():
    train_data = np.load(open('bottleneck_model_weights/bottleneck_features_train.npy','rb'))
    train_label = np.array(train_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))[:-10]
    
    test_data = np.load(open('bottleneck_model_weights/bottleneck_features_test.npy','rb'))
    test_label = np.array(test_df.eyewear.map({'Not_Glasses':0, 'Glasses':1}))[:-3]
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                 loss='binary_crossentropy',
                 metrics=['accuracy'])
    
    model.fit(train_data, train_label,
             epochs=50,
             batch_size=16,
             validation_data=(test_data,test_label))
    model.save_weights('bottleneck_model_weights/bottleneck_vgg_model.h5')

In [54]:
train_top_model()

Train on 512 samples, validate on 128 samples
Epoch 1/50
512/512 [==============================] - 0s 715us/step - loss: 1.4844 - acc: 0.6133 - val_loss: 0.5525 - val_acc: 0.6875
Epoch 2/50
512/512 [==============================] - 0s 199us/step - loss: 0.5770 - acc: 0.7188 - val_loss: 0.7599 - val_acc: 0.6406
Epoch 3/50
512/512 [==============================] - 0s 202us/step - loss: 0.4187 - acc: 0.8223 - val_loss: 0.8598 - val_acc: 0.6406
Epoch 4/50
512/512 [==============================] - 0s 195us/step - loss: 0.4532 - acc: 0.8008 - val_loss: 0.4884 - val_acc: 0.7812
Epoch 5/50
512/512 [==============================] - 0s 194us/step - loss: 0.2966 - acc: 0.8848 - val_loss: 0.5101 - val_acc: 0.7969
Epoch 6/50
512/512 [==============================] - 0s 190us/step - loss: 0.2718 - acc: 0.8945 - val_loss: 1.0055 - val_acc: 0.6250
Epoch 7/50
512/512 [==============================] - 0s 191us/step - loss: 0.2571 - acc: 0.8965 - val_loss: 0.4931 - val_acc: 0.7734
Epoch 8/50
512/5